First install dependencies:

pip install openmeteo-requests
pip install requests-cache retry-requests numpy pandas

In [7]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

In [9]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)
responses = []
df = pd.DataFrame()

In [30]:
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"

params = {
	"latitude": 51.5085,
	"longitude": -0.1257,
	"start_date": "1980-01-01",
	"end_date": "2020-01-01",
	"daily": ["weather_code",
           "temperature_2m_mean",
           "temperature_2m_max",
           "temperature_2m_min",
           "precipitation_sum",
           "rain_sum",
           "snowfall_sum",
           "precipitation_hours",
           "wind_speed_10m_max",
           "wind_gusts_10m_max",
           "wind_direction_10m_dominant",
           "shortwave_radiation_sum"]
}

#coordinates = {'madrid':{'latitud':'40,4165', 'longitude':'-3,7026'},
#               'london':{'latitud':'51,5085', 'longitude':'-0,1257'},
#               'paris':{'latitud':'48,8534', 'longitude':'2,3488'},
#               'berlin':{'latitud':'52,5244', 'longitude':'13,4105'},
#               'brussels':{'latitud':'50,8505', 'longitude':'4,3488'},
#               'istanbul':{'latitud':'41,0138', 'longitude':'28,9497'}
#}

In [31]:
def get_weather_data():
    response = openmeteo.weather_api(url, params=params)
    return response

In [32]:
def get_weather_dataframe():
    daily_dataframe = pd.DataFrame()
    response = get_weather_data()
    daily = response[0].Daily()
    daily_weather_code = daily.Variables(0).ValuesAsNumpy()
    daily_temperature_2m_mean = daily.Variables(1).ValuesAsNumpy()
    daily_temperature_2m_max = daily.Variables(2).ValuesAsNumpy()
    daily_temperature_2m_min = daily.Variables(3).ValuesAsNumpy()
    daily_precipitation_sum = daily.Variables(4).ValuesAsNumpy()
    daily_rain_sum = daily.Variables(5).ValuesAsNumpy()
    daily_snowfall_sum = daily.Variables(6).ValuesAsNumpy()
    daily_precipitation_hours = daily.Variables(7).ValuesAsNumpy()
    daily_wind_speed_10m_max = daily.Variables(8).ValuesAsNumpy()
    daily_wind_gusts_10m_max = daily.Variables(9).ValuesAsNumpy()
    daily_wind_direction_10m_dominant = daily.Variables(10).ValuesAsNumpy()
    daily_shortwave_radiation_sum = daily.Variables(11).ValuesAsNumpy()

    daily_data = {"date": pd.date_range(
        start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
        end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = daily.Interval()),
        inclusive = "left"
    )}

    daily_data["weather_code"] = daily_weather_code
    daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
    daily_data["temperature_2m_max"] = daily_temperature_2m_max
    daily_data["temperature_2m_min"] = daily_temperature_2m_min
    daily_data["precipitation_sum"] = daily_precipitation_sum
    daily_data["rain_sum"] = daily_rain_sum
    daily_data["snowfall_sum"] = daily_snowfall_sum
    daily_data["precipitation_hours"] = daily_precipitation_hours
    daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
    daily_data["wind_gusts_10m_max"] = daily_wind_gusts_10m_max
    daily_data["wind_direction_10m_dominant"] = daily_wind_direction_10m_dominant
    daily_data["shortwave_radiation_sum"] = daily_shortwave_radiation_sum

    temp_df = pd.DataFrame(data = daily_data)
    daily_dataframe = pd.concat([daily_dataframe,temp_df])
    return daily_dataframe


In [33]:
df = get_weather_dataframe()

In [34]:
display(df)

,date,weather_code,temperature_2m_mean,temperature_2m_max,temperature_2m_min,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum
0,1980-01-01 00:00:00+00:00,3.0,-1.929167,1.200000,-4.0000,0.000000,0.000000,0.00,0.0,16.375053,28.440001,320.117371,2.58
1,1980-01-02 00:00:00+00:00,3.0,-2.406250,1.050000,-4.5500,0.000000,0.000000,0.00,0.0,16.981165,29.519999,314.023834,3.99
2,1980-01-03 00:00:00+00:00,71.0,2.079167,8.900000,-4.5000,7.199999,7.199999,0.14,15.0,23.863409,45.000000,194.125839,0.69
3,1980-01-04 00:00:00+00:00,3.0,7.283333,9.900000,2.9500,0.000000,0.000000,0.00,0.0,24.063513,44.639996,229.569687,1.75
4,1980-01-05 00:00:00+00:00,3.0,3.785417,7.050000,1.9500,0.000000,0.000000,0.00,0.0,22.322901,38.880001,252.675827,3.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14606,2019-12-28 00:00:00+00:00,3.0,7.802333,9.656500,6.3065,0.000000,0.000000,0.00,0.0,13.708391,23.759998,184.041275,1.77
14607,2019-12-29 00:00:00+00:00,3.0,5.929417,8.856501,3.4065,0.000000,0.000000,0.00,0.0,15.937878,27.359999,203.607742,3.47
14608,2019-12-30 00:00:00+00:00,3.0,5.448166,10.056500,2.2065,0.000000,0.000000,0.00,0.0,16.575644,28.440001,234.087982,3.77
14609,2019-12-31 00:00:00+00:00,3.0,5.952333,9.256500,2.2065,0.000000,0.000000,0.00,0.0,16.981165,28.440001,84.347328,1.49


In [20]:
df.to_csv('weather_output.csv')

In [29]:
df.describe()

,weather_code,temperature_2m_mean,temperature_2m_max,temperature_2m_min,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum
count,1827.000000,1827.000000,1827.000000,1827.000000,1827.000000,1827.000000,1827.000000,1827.000000,1827.000000,1827.000000,1827.000000,1827.000000
mean,33.766830,11.110223,14.733046,7.392924,1.819376,1.780624,0.027433,3.723043,21.348982,40.225422,198.456619,10.555971
std,26.036232,5.488282,6.064040,5.182099,3.355421,3.291930,0.337926,4.708696,7.952250,14.691338,90.699081,7.392199
min,0.000000,-3.289334,-0.693500,-5.993500,0.000000,0.000000,0.000000,0.000000,5.692099,10.799999,0.399243,0.360000
25%,3.000000,6.888666,10.053250,3.406500,0.000000,0.000000,0.000000,0.000000,15.484185,29.519999,135.369247,4.050000
50%,51.000000,10.921084,14.300000,7.356500,0.300000,0.200000,0.000000,2.000000,20.211363,38.160000,222.883438,9.070000
75%,53.000000,15.561584,19.356501,11.453250,2.000000,1.900000,0.000000,6.000000,26.018548,48.599998,257.608948,16.469999
max,75.000000,27.521082,35.206501,19.656500,31.299999,31.299999,8.679998,23.000000,53.731731,100.799995,359.999969,29.490000
